# 유튜브 랭킹 데이터 수집하기

In [25]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [31]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [32]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [33]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [34]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [35]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [36]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip(' \n[]')
category

'음악/댄스/가수'

In [37]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [38]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6410만', '190억1809만', '371')

In [40]:
channels = []
for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip(' \n[]')
  subscriber = channel.select_one('.subscriber_cnt').text
  name = channel.select_one('.subject a').text.strip()
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.video_cnt').text[:-1]
  channels.append([category, subscriber, name, view, video])


In [41]:
df = pd.DataFrame(channels, columns=['카테고리', '구독자수','이름', '조회수', '비디오수'])
df

,카테고리,구독자수,이름,조회수,비디오수
0,음악/댄스/가수,6410만,BLACKPINK,190억1809만,371
1,음악/댄스/가수,6040만,HYBE LABELS,187억3584만,654
2,음악/댄스/가수,5650만,BANGTANTV,122억1297만,"1,579"
3,음악/댄스/가수,2850만,SMTOWN,218억9538만,"3,729"
4,키즈/어린이,2650만,Boram Tube Vlog [보람튜브 브이로그],110억5288만,223
...,...,...,...,...,...
95,음식/요리/레시피,408만,까니짱 [ G-NI ],10억3471만,482
96,키즈/어린이,408만,로미유 스토리[Romiyu Story],10억6621만,433
97,음악/댄스/가수,387만,NCT DREAM,3억7615만,229
98,BJ/인물/연예인,381만,허팝Heopop,31억3892만,"1,847"


##만과 억을 숫자로 바꿔주는 함수

In [42]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [43]:
convert_unit('10억5000만')

'1,050,000,000'

## 두 번째 페이지

In [44]:
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [45]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [46]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)
    results.append([category,name,subscriber,view,video])

In [47]:
df = pd.DataFrame(results, columns=['카테고리', '구독자수','채널명', '조회수', '비디오수'])
df

,카테고리,구독자수,채널명,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,100,000","19,018,090,000",371
1,음악/댄스/가수,HYBE LABELS,"60,400,000","18,735,840,000",654
2,음악/댄스/가수,BANGTANTV,"56,500,000","12,212,970,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,895,380,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...
95,음식/요리/레시피,까니짱 [ G-NI ],"4,080,000","1,034,710,000",482
96,키즈/어린이,로미유 스토리[Romiyu Story],"4,080,000","1,066,210,000",433
97,음악/댄스/가수,NCT DREAM,"3,870,000","376,150,000",229
98,BJ/인물/연예인,허팝Heopop,"3,810,000","3,138,920,000","1,847"


## page 1에서 10까지 크롤링하기

In [48]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [49]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,100,000","19,018,090,000",371
1,음악/댄스/가수,HYBE LABELS,"60,400,000","18,735,840,000",654
2,음악/댄스/가수,BANGTANTV,"56,500,000","12,212,970,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,895,380,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [50]:
df.to_csv('유튜브_순위.csv', index=None)

In [51]:
driver.close()